# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

### Mount Google Drive

The Google Colab environment is transient and will reset after any prolonged break in activity. To retain important and/or large files between sessions, use the following lines of code to mount your personal Google drive to this Colab instance:

In [45]:
try:
    #--- Mount gdrive to /content/drive/My Drive/
    from google.colab import drive
    drive.mount('/content/drive')
    
except: pass

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Throughout this assignment we will use the following global `MOUNT_ROOT` variable to reference a location to store long-term data. If you are using a local Jupyter server and/or wish to store your data elsewhere, please update this variable now.

In [0]:
# --- Set data directory
MOUNT_ROOT = '/content/drive/My Drive'

### Select Tensorflow library version

This assignment will use the (new) Tensorflow 2.0 library. Use the following line of code to select this updated version:

In [0]:
# --- Select Tensorflow 2.0 (only in Google Colab)
% tensorflow_version 2.x

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [48]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

### Imports

Use the following lines to import any additional needed libraries:

In [49]:
pip install keract


In [0]:
import os, numpy as np, pandas as pd
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers
from tensorflow.keras import backend as K
from tensorflow.keras import activations
from jarvis.train import datasets

# Data

As in the tutorial, data for this assignment will consist of the CIFAR-10 dataset comprising 10 different everyday objects (airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck). The following lines of code will:

1. Download the dataset (if not already present) 
2. Prepare the necessary Python generators to iterate through dataset
3. Prepare the corresponding Tensorflow Input(...) objects for model definition

In [51]:
# --- Download dataset
datasets.download(name='cifar')

# --- Prepare generators and model inputs
configs = {'batch': {'size': 256}}
gen_train, gen_valid, client = datasets.prepare(name='cifar', configs=configs)
inputs = client.get_inputs(Input)

[ 2020-04-18 04:59:37 ] [====================] 100.000% : Iterating | 000001    

# Training

In this assignment we will train a basic convolutional neural network on the CIFAR-10 dataset. At minumum you must include the following baseline techniques covered in the tutorial:

* convolutional operations
* batch normalization
* activation function
* subsampling

You are also **encouraged** to try different permuations and customizations to achieve optimal validation accuracy.

### Define the model

In [0]:
kwargs = {
    'kernel_size': (3, 3)
    }

conv = lambda x, filters, padding, kernel_size, strides: layers.Conv2D(filters = filters, padding = padding, kernel_size = kernel_size, strides = strides)(x)
norm = lambda x: layers.BatchNormalization()(x)
relu = lambda x: layers.LeakyReLU()(x)
maxpool = lambda x, pool_size : layers.MaxPooling2D(pool_size = pool_size, padding = 'same')(x)

In [0]:
conv1pool = lambda filters, x, padding, kernel_size : maxpool(relu(norm(conv(x,filters, padding, kernel_size,strides = 1))), pool_size = (2,2))
conv2pool = lambda filters, x, padding, kernel_size : maxpool(relu(norm(conv(x,filters, padding, kernel_size, strides = (2,2)))), pool_size = (2,2))

conv1 = lambda filters, x, padding, kernel_size : relu(norm(conv(x, filters, padding, kernel_size, strides=1)))
conv2 = lambda filters, x, padding, kernel_size : relu(norm(conv(x, filters, padding, kernel_size, strides=(2, 2))))

In [54]:
#--- Define model
l1 = conv1(96,inputs['dat'], 'same', (5,5))
l2 = conv1pool(96,l1, 'same', (5,5))
l3 = conv1(72,l2, 'same', (3,3))
l3_d3 = layers.Dropout(rate = 0.2)(l3)
l4 = conv1pool(48,l3_d3, 'same', (3,3))
l4_d4 = layers.Dropout(rate = 0.2)(l4)
l5 = conv1(36,l4_d4, 'valid', (3,3))
l5_d5 = layers.Dropout(rate = 0.2)(l5)
l6 = conv1pool(36,l5_d5, 'valid', (3,3))
f0 = layers.GlobalMaxPooling2D()(l6)


logits = {}
logits['class'] = layers.Dense(10, name='class')(f0)
# --- Create model
model = Model(inputs=inputs, outputs=logits)

optimizers.Adadelta(learning_rate = 5e-1, rho=0.99)
model.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dat (InputLayer)             [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 32, 32, 96)        7296      
_________________________________________________________________
batch_normalization_56 (Batc (None, 32, 32, 96)        384       
_________________________________________________________________
leaky_re_lu_56 (LeakyReLU)   (None, 32, 32, 96)        0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 32, 32, 96)        230496    
_________________________________________________________________
batch_normalization_57 (Batc (None, 32, 32, 96)        384       
_________________________________________________________________
leaky_re_lu_57 (LeakyReLU)   (None, 32, 32, 96)        0  

### Compile the model

In [0]:

# # --- Define model
# l1 = conv1pool(24,inputs['dat'])
# l2 = conv1(36,l1)
# l3 = conv1pool(48,l2)
# l4 = conv1(24,l3)

# f0 = layers.Flatten()(l4)

# d1 = layers.Dense(256, activation = 'relu')(f0)
# d1_0 = layers.Dropout(rate = 0.15)(d1)
# d2 = layers.Dense(64, activation = 'relu')(d1_0)
# d2_0 = layers.Dropout(rate = 0.15)(d2)

# logits = {}
# logits['class'] = layers.Dense(10, name='class')(d2_0)
# # --- Create model
# model = Model(inputs=inputs, outputs=logits)

# adam optimizer
# model.summary()

# 75%

#---------------------------------------------------------------------------

# # --- Define model
# l1 = conv1pool(32,inputs['dat'])
# l2 = conv1(40,l1)
# l3 = conv1pool(54,l2)
# l3_d3 = layers.Dropout(rate = 0.15)(l3)
# l4 = conv1pool(81,l3_d3)
# l4_d4 = layers.Dropout(rate = 0.15)(l4)
# l5 = conv1pool(122,l4_d4)
# f0 = layers.Flatten()(l5)

# d2 = layers.Dense(122)(f0)
# d2_relu = layers.LeakyReLU()(d2)
# d2_0 = layers.Dropout(rate = 0.15)(d2_relu)

# logits = {}
# logits['class'] = layers.Dense(10, name='class')(d2_0)
# # --- Create model
# model = Model(inputs=inputs, outputs=logits)

# model.summary()

# # --- Compile model
# model.compile(
#     optimizer=optimizers.Adadelta(learning_rate = 2e-1, rho=0.99),
#     loss={'class': losses.SparseCategoricalCrossentropy(from_logits=True)}, 
#     metrics={'class': 'sparse_categorical_accuracy'})

# 79% test accuracy, 90% training accuracy. Overfit

#---------------------------------------------------------------------------

# --- Define model
# l1 = conv1pool(32,inputs['dat'])
# l2 = conv1(42,l1)
# l3 = conv1pool(60,l2)
# l3_d3 = layers.Dropout(rate = 0.2)(l3)
# l4 = conv1pool(85,l3_d3)
# l4_d4 = layers.Dropout(rate = 0.2)(l4)
# l5 = conv1pool(110,l4_d4)
# f0 = layers.Flatten()(l5)

# d2 = layers.Dense(72, activation = 'relu')(f0)
# d2_0 = layers.Dropout(rate = 0.2)(d2)

# logits = {}
# logits['class'] = layers.Dense(10, name='class')(d2_0)
# # --- Create model
# model = Model(inputs=inputs, outputs=logits)

#optimizers.Adadelta(learning_rate = 5e-1, rho=0.99)
# model.summary()

# 80% test accuracy, 90% training accuracy. Overfit

#--------------------------------------------------

# # --- Define model

# #All Convolutional Net with dropout”
# l0 = layers.Dropout(rate = 0.2)(inputs['dat'])
# l1 = conv1(96,l0, 'same')
# l2 = conv1(96,l1, 'same')
# l3 = conv2(96,l2, 'same')
# l4 = conv1(192,l3, 'same')
# l4_d = layers.Dropout(rate = 0.15)(l4)
# l5 = conv1(192,l4_d, 'same')
# l5_d = layers.Dropout(rate = 0.15)(l5)
# l6 = conv2(192,l5_d, 'same')
# l6_d = layers.Dropout(rate = 0.15)(l6)
# l7 = conv1(192,l6_d, 'valid')
# l7_d = layers.Dropout(rate = 0.15)(l7)
# l8 = conv1(192,l7_d, 'valid')
# #l8_d = layers.Dropout(rate = 0.15)(l8)
# l9 = conv1pool(192,l8, 'valid')



# f0 = layers.Flatten()(l9)


# logits = {}
# logits['class'] = layers.Dense(10, name='class')(f0)
# # --- Create model
# model = Model(inputs=inputs, outputs=logits)

#optimizers.Adadelta(learning_rate = 5e-1, rho=0.99)
#model.summary()

# 80% test accuracy, 95% training accuracy. Overfit

In [0]:
# --- Compile model
model.compile(
    optimizer=optimizers.Adadelta(learning_rate = 1, rho=0.9999),
    loss={'class': losses.SparseCategoricalCrossentropy(from_logits=True)}, 
    metrics={'class': 'sparse_categorical_accuracy'})

### Train the model

In [57]:
model.fit(x = gen_train,
          steps_per_epoch = 250,
          epochs = 100,
          validation_data = gen_valid,
          validation_steps = 250,
          validation_freq = 4)

Epoch 1/100
250/250 [==============================] - 52s 209ms/step - loss: 1.5950 - sparse_categorical_accuracy: 0.4206
Epoch 2/100
250/250 [==============================] - 52s 206ms/step - loss: 1.1720 - sparse_categorical_accuracy: 0.5803
Epoch 3/100
250/250 [==============================] - 51s 202ms/step - loss: 0.9819 - sparse_categorical_accuracy: 0.6534
Epoch 4/100
250/250 [==============================] - 96s 385ms/step - loss: 0.8747 - sparse_categorical_accuracy: 0.6941 - val_loss: 1.0479 - val_sparse_categorical_accuracy: 0.6431
Epoch 5/100
250/250 [==============================] - 50s 202ms/step - loss: 0.8123 - sparse_categorical_accuracy: 0.7167
Epoch 6/100
250/250 [==============================] - 50s 200ms/step - loss: 0.7630 - sparse_categorical_accuracy: 0.7334
Epoch 7/100
250/250 [==============================] - 52s 207ms/step - loss: 0.7322 - sparse_categorical_accuracy: 0.7426
Epoch 8/100
250/250 [==============================] - 99s 394ms/step - loss: 

In [62]:
 test_train, test_valid = client.create_generators(test=True)
# --- Aggregate all example
xs = []
ys = []


for x,y in test_valid:
    xs.append(x['dat'])
    ys.append(y['class'])
    
xs = np.concatenate(xs)
ys = np.concatenate(ys)

[ 2020-04-18 07:07:49 ] [====================] 100.000% : Iterating | 012000    

In [63]:
# --- Predict
logits = model.predict(xs)
if type(logits) is dict:
    logits = logits['class'] # take out class
pred = np.argmax(logits, axis = 1)

df = pd.DataFrame(index=np.arange(pred.size))

# --- Define columns
df['true'] = ys[:, 0]
df['pred'] = pred
df['corr'] = df['true'] == df['pred']

# --- Print accuracy
print(df['corr'].mean())


# --- Argmax

0.7989166666666667


# --- Predict
logits = model.predict(xs)
if type(logits) is dict:
    logits = logits['class'] # take out class
pred = np.argmax(logits, axis = 1)

df = pd.DataFrame(index=np.arange(pred.size))

# --- Define columns
df['true'] = ys[:, 0]
df['pred'] = pred
df['corr'] = df['true'] == df['pred']

# --- Print accuracy
print(df['corr'].mean())

75%

# --- Argmax

**Note**: this cell is used only to check for model performance. It will not be graded. Once you are satisfied with your model, proceed to submission of your assignment below.

### Results

When ready, create a `*.csv` file with your compiled **validation** cohort statistics. There is no need to submit training performance accuracy. As in the tutorial, ensure that there are at least three columns in the `*.csv` file:

* true (ground-truth)
* pred (prediction)
* corr (correction prediction, True or False)

In [0]:
# --- Create *.csv


                              
# --- Serialize *.csv
fname = '{}/models/cnn/results.csv'.format(MOUNT_ROOT)
os.makedirs(os.path.dirname(fname), exist_ok=True)
df.to_csv('resultsAssignment3.csv')